In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
# Load parsed reviews into a DataFrame
df = session.table("shipping_logs")

# Show a sample of the data
df.show()

In [ ]:
from snowflake.snowpark.functions import col, count
shipment_counts = (
    df.group_by(col('"Shipping_Date"'))
      .agg(count('"Order_ID"').alias('"Shipment_Count"'))
      .sort('"Shipping_Date"')
)
shipment_counts.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
shipment_df = shipment_counts.to_pandas()
shipment_df.plot(x="Shipping_Date", y="Shipment_Count", kind="line")
plt.title("Shipments Per Day")
plt.xlabel("Date")
plt.ylabel("Number of Shipments")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()